<a href="https://colab.research.google.com/github/bhavya26082004/python/blob/main/AD4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install emot

In [31]:

text1= "What are you saying 😂, I am the boss 😎, and why are you so 😔"

In [32]:
import re
from emot.emo_unicode import UNICODE_EMOJI
from emot.emo_unicode import EMOTICONS_EMO

In [33]:
def converting_emojis(text):
  for emot in UNICODE_EMOJI:
    text=text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split()))
  return text
converting_emojis(text1)

'What are you saying face_with_tears_of_joy, I am the boss smiling_face_with_sunglasses, and why are you so pensive_face'

In [34]:
def emoji_removal(string):
    emoji_unicodes = re.compile("["
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
        u"\U00002500-\U00002BEF"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"
        u"\u3030"
    "]+", flags=re.UNICODE)

    return emoji_unicodes.sub(r'', string)
emoji_removal(text1)

'What are you saying , I am the boss , and why are you so '

In [35]:
sent = ["What are you saying 😂", "I am the boss 😎", "why are you so 😔"]
import pandas as pd
df = pd.DataFrame({'text': sent})
df.head()


,text
0,What are you saying 😂
1,I am the boss 😎
2,why are you so 😔


In [36]:
import pandas as pd
dataset = pd.read_csv('/content/hate_speech.csv')
dataset.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [37]:
dataset.shape

(5242, 3)

In [38]:
dataset.label.value_counts()

,count
label,
0,3000
1,2242


In [39]:
for index, tweet in enumerate(dataset["tweet"][10:15]):
    print(index+1,".",tweet)

1 .  â #ireland consumer price index (mom) climbed from previous 0.2% to 0.5% in may   #blog #silver #gold #forex
2 . we are so selfish. #orlando #standwithorlando #pulseshooting #orlandoshooting #biggerproblems #selfish #heabreaking   #values #love #
3 . i get to see my daddy today!!   #80days #gettingfed
4 . ouch...junior is angryð#got7 #junior #yugyoem   #omg 
5 . i am thankful for having a paner. #thankful #positive     


In [40]:
import re
def clean_text(text):
    text = re.sub(r'[^a-zA\']', '', text)
    text = re.sub(r'[^\\x00-\x7F]+', ' ', text)
    text = text.lower()
    return text

In [41]:
dataset['clean_text'] = dataset.tweet.apply(lambda x: clean_text(x))

In [42]:
dataset.head(10)

,id,label,tweet,clean_text
0,1,0,@user when a father is dysfunctional and is s...,userwhenafatherisdysfunctionalandissoselfishhe...
1,2,0,@user @user thanks for #lyft credit i can't us...,useruserthanksforlyftcreditican tusecausetheyd...
2,3,0,bihday your majesty,bihdayyourmajesty
3,4,0,#model i love u take with u all the time in ...,modeliloveutakewithuallthetimeinur
4,5,0,factsguide: society now #motivation,factsguidesocietynowmotivation
5,6,0,[2/2] huge fan fare and big talking before the...,hugefanfareandbigtalkingbeforetheyleavechaosan...
6,7,0,@user camping tomorrow @user @user @user @use...,usercampingtomorrowuseruseruseruseruseruseruse...
7,8,0,the next school year is the year for exams.ð...,thenextschoolyearistheyearforexamscan tthinkab...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...,wewonlovethelandallincavschampionsclevelandcle...
9,10,0,@user @user welcome here ! i'm it's so #gr...,useruserwelcomeherei mit ssogr


In [43]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [44]:
from nltk.corpus import stopwords
len(stopwords.words('english'))

179

In [45]:
stop = stopwords.words('english')

In [46]:
def gen_freq(text):
    word_list = []
    for tw_words in text.split():
        word_list.extend(tw_words)
    word_freq = pd.Series(word_list).value_counts()
    word_freq = word_freq.drop(stop, errors='ignore')
    return word_freq

In [47]:
def any_neg(words):
  for word in words:
    if word in ['n', 'no', 'non', 'not'] or re.search(r"\wn't", word):
      return 1
    else:
      return 0

In [48]:
def any_rare(words, rare_100):
  for word in words:
    if word in rare_100:
      return 1
    else:
      return 0

In [49]:
def is_question(words):
  for word in words:
    if word in ['when', 'what', 'how', 'why', 'who']:
      return 1
    else:
      return 0

In [50]:
word_freq=gen_freq(dataset.clean_text.str)
rare_100=word_freq[-100:]
dataset['word_count']=dataset.clean_text.str.split().apply(lambda x: len(x))
dataset['any_neg']=dataset.clean_text.str.split().apply(lambda x: any_neg(x))
dataset['any_rare']=dataset.clean_text.str.split().apply(lambda x: any_rare(x, rare_100))
dataset['is_question']=dataset.clean_text.str.split().apply(lambda x: is_question(x))
dataset.clean_text.apply(lambda x: len(x))
dataset['char_count']=dataset.clean_text.apply(lambda x: len(x))

In [51]:
dataset.head(10)

,id,label,tweet,clean_text,word_count,any_neg,any_rare,is_question,char_count
0,1,0,@user when a father is dysfunctional and is s...,userwhenafatherisdysfunctionalandissoselfishhe...,1,0,0,0,79
1,2,0,@user @user thanks for #lyft credit i can't us...,useruserthanksforlyftcreditican tusecausetheyd...,3,0,0,0,95
2,3,0,bihday your majesty,bihdayyourmajesty,1,0,0,0,17
3,4,0,#model i love u take with u all the time in ...,modeliloveutakewithuallthetimeinur,1,0,0,0,34
4,5,0,factsguide: society now #motivation,factsguidesocietynowmotivation,1,0,0,0,30
5,6,0,[2/2] huge fan fare and big talking before the...,hugefanfareandbigtalkingbeforetheyleavechaosan...,1,0,0,0,88
6,7,0,@user camping tomorrow @user @user @user @use...,usercampingtomorrowuseruseruseruseruseruseruse...,1,0,0,0,52
7,8,0,the next school year is the year for exams.ð...,thenextschoolyearistheyearforexamscan tthinkab...,2,0,0,0,105
8,9,0,we won!!! love the land!!! #allin #cavs #champ...,wewonlovethelandallincavschampionsclevelandcle...,1,0,0,0,61
9,10,0,@user @user welcome here ! i'm it's so #gr...,useruserwelcomeherei mit ssogr,3,0,0,0,30


In [52]:
from sklearn.model_selection import train_test_split

X = dataset[['word_count', 'any_neg', 'any_rare', 'char_count', 'is_question']]
y = dataset.label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=0)

In [53]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model = model.fit(X_train, y_train)
pred = model.predict(X_test)

In [54]:
model.predict(X_test[5:10])

array([0, 0, 0, 0, 0])

In [56]:
from sklearn.metrics import accuracy_score
print("Accuracy : ", accuracy_score(y_test, pred)*100, "%")

Accuracy :  57.292659675881794 %


In [58]:
from sklearn.ensemble import RandomForestClassifier
clf_rf=RandomForestClassifier()

clf_rf.fit(X_train,y_train)

rf_pred=clf_rf.predict(X_test).astype(int)

In [60]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,rf_pred))
print(classification_report(y_test,rf_pred))
print('Accuracy: ', accuracy_score(y_test,rf_pred))

[[403 196]
 [234 216]]
              precision    recall  f1-score   support

           0       0.63      0.67      0.65       599
           1       0.52      0.48      0.50       450

    accuracy                           0.59      1049
   macro avg       0.58      0.58      0.58      1049
weighted avg       0.59      0.59      0.59      1049

Accuracy:  0.5900857959961868


In [61]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(class_weight='balanced')
logreg.fit(X_train, y_train)

LogisticRegression(class_weight='balanced')

In [62]:
y_pred = logreg.predict(X_test)

In [63]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.66      0.56      0.61       599
           1       0.51      0.62      0.56       450

    accuracy                           0.58      1049
   macro avg       0.59      0.59      0.58      1049
weighted avg       0.60      0.58      0.59      1049

